# **Este *ipynb* está enmarcado dentro de un trabajo que busca predecir qué alumnos del departamento de Sistemas la UTN FRBA desertarán.**

Los datos disponibles se presentan en tres archivos separados.

Los registros de estas tres tablas se encuentran relacionados a través de un ID anonimizado que corresponde a un estudiante en particular. 

En este **ipynb** se preprocesan los datos disponibles hasta llegar a unificarlos en una sola tabla, la cual denominaremos **merged_df**.


## **Aclaraciones**

Aquellas líneas de código identificadas con **" # "** fueron extraídas de la siguente fuente:
* https://github.com/sebajarem/Analisis_desercion_en_ingenieria/tree/master/desercion/munge/00_datos_01

Aquellas líneas de código identificadas con **" ## "** son de elaboración propia.

## **Definiciones**

* **Actividad​**: Se considera que un estudiante tiene ​actividad en un año determinado si: tiene alguna materia con el campo “Tipo de aprobación” distinto de “libre” ó si rindió algún final de esa u otra materia. 
 *Notar que ​actividad es una característica siempre asociada a un año determinado ​XX​.*
---
* (ALF) **Año Lectivo Final**​: Año que se toma como punto de partida para analizar HACIA ATRÁS las historias disponibles hasta 2008. En nuestro caso, por ahora, **ALF = 2016**. 
---
* (ALI) **Año Lectivo Inicial**: Año a partir de que contamos con datos. En nuestro caso, por ahora; **ALI = 2008**. 
---
* (AUA) **Año de Última Actividad​**: Último año en que el estudiante haya tenido alguna actividad​. 
---
* (AI) **Año de Ingreso**​: Año en el que el estudiante haya realizado su primera actividad.
---
* **Alumno ​activo a la fecha ​XX**​: Se considera que un estudiante está “activo a la fecha ​XX​” si en el año ​XX ​registró ​actividad​.  
---
* (AE) **Año de egreso**​: Año en que aprobó “Proyecto Final”.  
---
* **Condición de Egresado​**: La condición de ​egresado se puede asignar a un alumno en el año de egreso o posteriores. *Ver que se necesita indicar un año para ver si pertenece a esa categoría.* 
---
* **Condición de Desertor**​: Se considera que un alumno es desertor para el año ​XX si **no** registra actividad ni en el año ​XX ni en el año ​XX - 1​. *Ver que se necesita indicar un año para ver si pertenece a esa categoría.*
---

## **Importación de librerías**





In [ ]:
## Importamos librerías para manipulación de datos.
import pandas as pd
import numpy as np

In [ ]:
from sklearn.preprocessing import LabelEncoder

## **Defino años para el análisis**

In [ ]:
#
ALF = 2016 ## Año lectivo final
ALI = 2008 ## Año lectivo inicial

#Seteo el ano sobre el que voy a medir
ano = 2017

## **Google Colaboratory o Local**
El notebook podrá ser corrido tanto localmente como en Google Colaboratory.

El usuario deberá modificar el root path de acuerdo a su conveniencia.


In [ ]:
## Verificamos si estamos corriendo el noteboock en Google Colaboratory.
var_google_colab = 'google.colab' in str(get_ipython())
print(var_google_colab)

## En el caso de estar en Google Colab, montamos nuestro Drive.
if var_google_colab:
  from google.colab import drive
  drive.mount('/content/gdrive',force_remount=True)
  ## Direccion root donde está el notebook.
  root_path = "/content/gdrive/MyDrive/Colab Notebooks/GIAR/"

## En el caso de no estar en Google Colab, estamos corriendo localmente el notebook.
else:
  root_path = ""

## **Datasets**

In [ ]:
## Importamos el dataset Datos-Alumnos-SIGA.
alumnos = pd.read_csv(root_path + 'datos/alumnos_desertores.csv')

In [ ]:
## Importamos el dataset Cursadas-Alumnos-SIGA.
cursadas = pd.read_csv(root_path + 'datos/Cursadas-Alumnos-SIGA.csv')

In [ ]:
## Importamos el dataset Finales-Alumnos-SIGA.
finales = pd.read_csv(root_path + 'datos/Finales-Alumnos-SIGA.csv')

## **PREPROSESAMIENTO DE LOS DATOS**

### **ALUMNOS** ###

#### Variables que se decidieron sacar:
- **Pais**
- **Estado Civil**


#### Variables que se van a generar:
- **EsTecnico** como una categorica que es "SI" en el caso de que sea tecnico y "NO" en otro caso. Se deben respetar los nulos.
- **Distancia** como una numérica que medirá la distancia en KM que existe entre el domicilio declarado del alumno y la universidad. Dado que la FRBA cuenta con dos sedes, se fijará como locación de la misma el punto medio entre ambas sedes. En este caso, Av. Directorio 1150, CABA.
En caso que la distancia sea mayor a 70 KM, se supondrá que el dato del domicilio está desactualizado y se definirá un tratamiento particular para dicho registro. Se deben respetar los nulos.

In [ ]:
## Eliminamos aquellos registros en los que no se cuenta con información.  
## alumnos.dropna(subset = ['Estudios Secundarios'], how ='any', inplace = True)

In [ ]:
#Genero la nueva columna de esTecnico
alumnos['EsTecnico']=alumnos.apply(lambda row: 1 if row['Estudios Secundarios']=='Técnico'  else (row['Estudios Secundarios'] if pd.isnull(row['Estudios Secundarios']) else 0), axis=1)

In [ ]:
## Generamos el dataset sin las variables que se decidieron.
alumnos_df = alumnos[['Codigo Alumno','EsTecnico','deserto']]

### **CURSADAS** ###

#### Variables que se decidieron sacar:
- Curso
- Materia
- Departamento
- Ciclo Lectivo (la vamos a sacar pero sera utilizada para filtrar los registros)
- Modalidad


#### Variables que se van a generar:
- Edad al ingreso (Ano ingreso - Ano nacimiento) 
- Turno una columna por turno con la cantidad 
- Tipo de aprobación por cada una de las variables la cantidad 
- Maximo recursada por materias (existe mas de un registro por alumno y materia)
- Cantidad de veces recursada regular OK
- Descripción de recursada regular OK
- Edad en ano de ultima actividad

Tener en cuenta que se debe guardar el ciclo lectivo mas actual

In [ ]:
#Filtro el set de datos por el año en que lo quiero medir
cursadas=cursadas.loc[cursadas['Ciclo Lectivo de Cursada']<ano-2,:]

In [ ]:
## Vemos como se distribuye la variable 'Modalidad'.
cursadas['Modalidad'].value_counts()

In [ ]:
## Unificamos la denominación para hacer referencia a cursadas anuales.
cursadas['Modalidad'].replace('     1/1','Anual', inplace = True)

In [ ]:
## Identificamos nombres de materias poco frecuentes.
a = pd.DataFrame(cursadas['Materia'].value_counts())
a[a<20].dropna(how='any')

In [ ]:
## Unificamos la denominación en aquellos casos en los que se identificó que existían diferentes nombres o abreviaturas para hacer referencia a una misma materia.
cursadas['Materia'].replace('Tecnologías Avanzadas en la Construcción de Sw','Tecnologías Avanzadas en la Construcción de Software', inplace = True)
cursadas['Materia'].replace('Introducción a la Ingeniería en Software','Ingeniería en Software', inplace = True)
cursadas['Materia'].replace('Inglés (Comunicaciónal )','Inglés Comunicaciónal I', inplace = True)
cursadas['Materia'].replace('Sistemas de Información Geografica','Sistemas de Información Geografico', inplace = True)
cursadas['Materia'].replace('Autocad - Sistemas de Representación','Sistemas de Representación', inplace = True)

In [ ]:
#Genero la variable edad al ingreso
cursadas['edad al ingreso']=cursadas['Año de ingreso']-cursadas['Año de nacimiento']

In [ ]:
## Generamos la variable año_alumno_uni que representa en que año de su carrera cursó el alumno la materia. 0 = primer año (ingresante), 1 = segundo año, 2 = tercer año... 
cursadas['año_alumno_uni'] = cursadas['Ciclo Lectivo de Cursada'] - cursadas['Año de ingreso']

In [ ]:
## Generamos la varaiable 'cursada' que permita identificar a todas las cursadas iguales independientemente del alumno que las curse.
cursadas['cursada'] = cursadas.apply(lambda row: (str(row['Ciclo Lectivo de Cursada'])+row['Curso']+row['Modalidad']+row['Turno']+row['Materia']), axis = 1)

In [ ]:
cursadas['cursada'].value_counts()

In [ ]:
## Asignamos a cada alumno un grupo de ingreso basándonos en su cursada de la materia integradora de primer nivel durante su año de ingresante.
cursadas['grupo_ingreso_nivel1'] = cursadas.apply(lambda row: (str(row['Ciclo Lectivo de Cursada'])+' '+row['Curso']+row['Modalidad']+' '+row['Turno']) if (row['Materia']=='Sistemas y Organizaciones' and row['año_alumno_uni']==0)  else np.nan, axis=1)

In [ ]:
## Genero un dataset en el que se guarda información referida la cursada de la materia integradora de 1er nivel.
cursadas_integradoras = cursadas[['Codigo Alumno','grupo_ingreso_nivel1']]
cursadas.drop(columns=['grupo_ingreso_nivel1'], inplace = True)

In [ ]:
## Eliminamos los registros de cursadas que no están relacionadas a la materia integradora de primer nivel cursada en primer año.
cursadas_integradoras = cursadas_integradoras.dropna(how='any')

## Agrupo las cursadas por alumno y para aquellos casos en que el alumno haya más de un registro, decidimos quedarnos con la primera cursada.
## Los nombres de las cursadas comienzan con el año, de allí que se pueda usar min().
cursadas_integradoras = cursadas_integradoras.groupby('Codigo Alumno').min()
cursadas_integradoras.shape

In [ ]:
cursadas['Codigo Alumno'].value_counts().count()

In [ ]:
cursadas_integradoras['grupo_ingreso_nivel1'].isnull().value_counts()

In [ ]:
## Descartaremos aquellos grupos con menos de 5 alumnos registrados en todo el dataframe de cursadas.
grupos_descartar = pd.DataFrame((cursadas_integradoras['grupo_ingreso_nivel1'].value_counts() < 5))
grupos_descartar = grupos_descartar.loc[grupos_descartar['grupo_ingreso_nivel1']==True]
grupos_descartar = list(grupos_descartar.index)

cursadas_integradoras['grupo_ingreso_nivel1'].replace(grupos_descartar, np.nan, inplace = True)
cursadas_integradoras.dropna(how='any',inplace=True)

In [ ]:
## Transformo la variable en numérica.
## Cada curso particular tendrá un número asignado.
le = LabelEncoder()
cursadas_integradoras['grupo_ingreso_nivel1'] = le.fit_transform(cursadas_integradoras['grupo_ingreso_nivel1'])

In [ ]:
#Genero la lista de las descripciones de recursadas
recursadas_df=cursadas.groupby(['Cantidad de veces recursada regular','Descripción de recursada regular'])['Materia'].max().reset_index()[['Cantidad de veces recursada regular','Descripción de recursada regular']]

#Agrupo las cursadas de cada alumno por materia y calculo la mayor cantidad de recursadas y lo uno con la descripcion de cada una
cursadas_df=cursadas.groupby(['Codigo Alumno','Materia'])['Cantidad de veces recursada regular'].max().reset_index().join(recursadas_df.set_index('Cantidad de veces recursada regular'),on='Cantidad de veces recursada regular',rsuffix='_o')

#Setea la descripcion como categoria
cursadas_df['Descripción de recursada regular']=cursadas_df['Descripción de recursada regular'].astype('category')

#Elimino la columna materia, y agrupo por alumno y sumarizo las descripciones
recursadas_df=pd.get_dummies(cursadas_df,columns=['Descripción de recursada regular']).drop(['Materia'], axis=1).groupby(['Codigo Alumno']).sum()

In [ ]:
#Chequeo que las columnas sean las que necesito
recursadas_df.head()

In [ ]:
recursadas_df['Descripción de recursada regular_Recurso n Veces (>5)'].value_counts()

In [ ]:
#Genero las dummies que se acordaron y los valores restantes
cursadas_df=pd.get_dummies(cursadas,columns=['Turno','Tipo de aprobación']).groupby(['Codigo Alumno']).agg({'Ciclo Lectivo de Cursada':['max'],
        'edad al ingreso':['max'],
        'Año de ingreso':['max'], 
        'Turno_Mañana':['sum'], 'Turno_Noche':['sum'], 'Turno_Tarde':['sum'], 
        'Tipo de aprobación_Cambio Curso':['sum'],
        'Tipo de aprobación_Firmo':['sum'],
        'Tipo de aprobación_Libre':['sum'], 
        'Tipo de aprobación_No Firmo':['sum'],
        'Tipo de aprobación_Promociono':['sum'],
        'Sexo':['max']})

#Corrijo los nombres de las columnas
cursadas_df.columns = [col[0] for col in cursadas_df.columns.values]

#Uno este DataFrame con el calculado con las descripciones de recursadas y lo guardo en el dataframe de cursadas
cursadas_df=cursadas_df.reset_index().join(recursadas_df,on='Codigo Alumno')

In [ ]:
cursadas_df.head()

In [ ]:
## Unimos el DataFrame con el que contiene el grupo de ingreso y lo guardo.
cursadas_df=cursadas_df.join(cursadas_integradoras,on='Codigo Alumno')

In [ ]:
cursadas_df['grupo_ingreso_nivel1'].isnull().value_counts()

In [ ]:
cursadas_df.head()

### **FINALES** ###

#### Columnas a sacar ####
- Materia
- Ano (Solo se usa para filtrar)


#### Variables a generar
- Promedio sobre los maximos de Nota
- Promedio sobre la nota (con aplazos)
- Cantidad de veces que aprobo
- Cantidad de veces que no aprobo
- Cantidad de veces que promociono

In [ ]:
#Filtro el set de datos por el ano en que lo quiero medir
finales=finales.loc[finales['Año']<ano-2,:]

In [ ]:
## Identificamos nombres de materias poco frecuentes.
a = pd.DataFrame(finales['Materia'].value_counts())
a[a<10].dropna(how='any')

In [ ]:
## Unificamos la denominación en aquellos casos en los que se identificó que existían diferentes nombres o abreviaturas para hacer referencia a una misma materia.
finales['Materia'].replace('Física','Física I', inplace = True)
finales['Materia'].replace('Arquitectura de Computadores','Arquitectura de Computadoras', inplace = True)

In [ ]:
##
finales['Nota'].value_counts()

In [ ]:
## Elimino los registros en los que haya notas con 0.
## finales = finales[finales.Nota != 0]

In [ ]:
#Corrijo los finales para que no haya notas con 11
finales.loc[finales['Nota']==11,['Nota']]=10
#Genero la columna de no aprobado
finales['noAprobado']=finales['Aprobado'].apply(lambda row: 1 if row==0 else 0)
#Agrupo los finales y me quedo con el promedio de las notas maximas
finales_max_df=finales.groupby(['Codigo Alumno','Materia'])['Nota'].max().reset_index().groupby(['Codigo Alumno'])['Nota'].mean().reset_index()

#Seteo el nombre a las columnas para que no se me superpongan
finales_max_df.columns=['Codigo Alumno','Nota_max_prom']

#Genero el resto de las agrupaciones sobre el set de finales
finales_df=finales.groupby(['Codigo Alumno']).agg({'noAprobado':[sum],'Aprobado':[sum],'Promociono':[sum],'Nota':['mean'],'Año':['max']})

#Corrijo los nombres de las columnas
finales_df.columns = [col[0] for col in finales_df.columns.values]

#Le agrego a los finales la columna de finales maximos agrupados
finales_df=finales_df.reset_index().join(finales_max_df.set_index('Codigo Alumno'),on='Codigo Alumno')

In [ ]:
## Agrego la columna 'Indice_aprobación', la cual se calcula como la cantidad de finales aprobados sobre el total de finales rendidos. 
finales_df['Indice_aprobacion'] = (finales_df['Aprobado'])/(finales_df['Aprobado']+finales_df['noAprobado'])

In [ ]:
finales_df.head()

### **AGRUPACIÓN de todos los registros** ###

#### merged_df

In [ ]:
#
merged_df = alumnos_df.join(cursadas_df.set_index('Codigo Alumno'),on='Codigo Alumno').join(finales_df.set_index('Codigo Alumno'),on='Codigo Alumno')

merged_df = merged_df[(merged_df['Codigo Alumno'].isin(finales_df['Codigo Alumno'])) & (merged_df['Codigo Alumno'].isin(cursadas_df['Codigo Alumno']))]

In [ ]:
## Generamos la variable 'cantidad de años', la cual indica la cantidad de años transcurridos desde que el alumno se inscribió hasta su última actividad registrada.
merged_df['cantidad de años'] = merged_df.apply(lambda row: (row['Año'] - row['Año de ingreso']) if (row['Año']>row['Ciclo Lectivo de Cursada'])  else (row['Ciclo Lectivo de Cursada'] - row['Año de ingreso']), axis=1)

In [ ]:
## Nos desprendemos de la variable 'Año de ingreso'.
merged_df.drop(['Año de ingreso'], axis=1, inplace = True)
merged_df.shape

In [ ]:
#Guardo los registros en un csv
merged_df.to_csv(root_path + 'datos/baseline_2009_02_00.csv',index=False)

In [ ]:
#
merged_df['deserto'].value_counts()

In [ ]:
merged_df.head()

In [ ]:
merged_df['grupo_ingreso_nivel1'].isnull().value_counts()